In [48]:
import mbuild as mb
import numpy as np
import mbuild.lib.molecules.water as water_models

## Problem: 

mBuild Compounds provide a wrapper to openbabel to enable energy minimization, which will by default use the UFF forcefield.  

This can be problematic for a rigid molecule like water, as the energy minimization will result in changes to the angle/distances of the atoms in each water molecules, which will not agree with the ultimate force field used in the simulation.

While openbabel does support constraints (e.g., bonds and angles), these will not necessarily be exact as the constraints are just really strong harmonic bonds/angles. 

Let us generate a box of water, energy minimize and look at the distances and angles.

In [49]:
# create a water box
box_temp = mb.Box([1.0, 1.0, 1.0])

# pack a box with water
water_system = mb.fill_box(water_models.WaterSPC(), density=1000, overlap=0.22,
               edge=0.15, sidemax=box_temp.Lz+1, box=box_temp, seed=1234)


In [50]:
# energy minimize
water_system.energy_minimize()

In [51]:
water_system.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [53]:
water_system.save('config1.mol2', overwrite=True)

In [54]:
def angle_between(v1, v2):
    v1_u = v1/np.linalg.norm(v1)
    v2_u = v2/np.linalg.norm(v2)
    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))*180.0/np.pi

#now let us see how messed up the angles and distances are after energy minimizing with UFF
angles = []
dists = []
for water in water_system.children:
    #the first particle is oxygen in the molecule
    v1 = water.children[1].pos - water.children[0].pos
    v2 = water.children[2].pos - water.children[0].pos
    dists.append(np.linalg.norm(v1))
    dists.append(np.linalg.norm(v2))
    angles.append(angle_between(v1,v2))

Let us look at the mean values for the angles and bond distances, which we note for SPC are 109.47 deg and 0.1 nm.

In [55]:
print(np.array(angles).mean().round(4), " deg")
print(np.array(dists).mean().round(6), " nm")

104.5402  deg
0.098995  nm


## Solution

We can use force_overlap routine (with add_bond=False) to translate and orient a copy of our original water model to each of the waters in the energy minimized configuration.  

We can either create a new system and add each of these reoriented/translated water molecules (with correct angdles/distances) or just reassign the coordinates in the original compound.  Below I do the second approach. 

In [56]:
spc = water_models.WaterSPC()
for water in water_system.children:

    temp = mb.clone(spc)
    mb.force_overlap(temp, temp, water, add_bond=False)
    water.xyz = temp.xyz
    


In [57]:
water_system.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

We can then check the angles and distances and we will see that we are exact again, and in a reasonable energy minimized configuration

In [58]:
#now let us see the angles and distances back to expected
angles = []
dists = []
for water in water_system.children:
    #the first particle is oxygen in the molecule
    v1 = water.children[1].pos - water.children[0].pos
    v2 = water.children[2].pos - water.children[0].pos
    dists.append(np.linalg.norm(v1))
    dists.append(np.linalg.norm(v2))
    angles.append(angle_between(v1,v2))

In [59]:
print(np.array(angles).mean().round(4), " deg")
print(np.array(dists).mean().round(6), " nm")



109.47  deg
0.1  nm


In [60]:
water_system.save('config2.mol2', overwrite=True)